In [ ]:
!pip install pymupdf reportlab

In [ ]:
!pip install PyPDF2

## Презентации в формате .pdf собранные вручную

In [23]:
raw_path = "/content/raw_data_1"

Проверка и очистка данных

In [24]:
import os
from PyPDF2 import PdfReader

def check_pdf_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            try:
                path = os.path.join(directory, filename)
                PdfReader(path)  # Пробуем открыть файл
                print(f"{filename}: OK")
            except Exception as e:
                print(f"{filename}: Error - {e}")

check_pdf_files(raw_path)


2426HG2RXRZNRHX3YHORDF5C57PPAAHE (1).pdf: OK
2NYYOQN3UKYWMKXDAMTWP4ZNMGBBTLCA.pdf: OK
business-policy.pdf: OK
2IOECB7XKI56SDSRDXHXTAUDGCMYTVI7.pdf: OK
2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf: OK
2C45NWO7FMTDRTBBR6FA442OZKSCLGZZ.pdf: OK
2426HG2RXRZNRHX3YHORDF5C57PPAAHE (1) (1).pdf: OK
25OOF5WZSJ4M6JG3BPYDO56GYBR65WET.pdf: OK
27CRLC3DDVOHBJ27UZSHYXRO3OB4BHIY.pdf: OK


Удаление пропусков и пустых слайдов

In [25]:
import fitz  # PyMuPDF

def check_empty_slides(directory):
    empty_slides = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            path = os.path.join(directory, filename)
            doc = fitz.open(path)

            for page_num in range(len(doc)):
                page = doc[page_num]
                text = page.get_text()
                if not text.strip():  # Если текст пустой
                    empty_slides.append((filename, page_num + 1))

    if empty_slides:
        print("Пустые слайды:")
        for slide in empty_slides:
            print(f"Файл: {slide[0]}, Слайд: {slide[1]}")
    else:
        print("Пустых слайдов не найдено.")

check_empty_slides(raw_path)


Пустые слайды:
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 21
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 22
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 23
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 24
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 25


Выбросы

In [26]:
def check_image_only_pages(directory):
    image_only_pages = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            path = os.path.join(directory, filename)
            doc = fitz.open(path)

            for page_num in range(len(doc)):
                page = doc[page_num]
                text = page.get_text()

                # Проверим, есть ли текст или только изображения
                if not text.strip() and len(page.get_images(full=True)) > 0:
                    image_only_pages.append((filename, page_num + 1))

    if image_only_pages:
        print("Страницы с изображениями, без текста:")
        for page in image_only_pages:
            print(f"Файл: {page[0]}, Слайд: {page[1]}")
    else:
        print("Нет страниц с изображениями без текста.")

check_image_only_pages(raw_path)


Страницы с изображениями, без текста:
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 21
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 22
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 23
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 24
Файл: 2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf, Слайд: 25


Отчет об ошибках

In [27]:
import json

def generate_clean_report(directory):
    report = []
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            path = os.path.join(directory, filename)
            doc = fitz.open(path)
            file_report = {"file": filename, "empty_pages": []}

            for page_num in range(len(doc)):
                page = doc[page_num]
                text = page.get_text()
                if not text.strip():
                    file_report["empty_pages"].append(page_num + 1)

            if file_report["empty_pages"]:
                report.append(file_report)
    print(report)
    # Сохранить отчет в JSON
    with open("cleaning_report.json", "w") as f:
        json.dump(report, f, indent=4)

generate_clean_report(raw_path)


[{'file': '2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf', 'empty_pages': [21, 22, 23, 24, 25]}]


 Изменение имени файлов

In [28]:
import os
import re
from datetime import datetime

def rename_presentation_files(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".pdf"):
            # Получаем полный путь
            old_path = os.path.join(directory, filename)

            # Убираем расширение файла
            base_name = os.path.splitext(filename)[0]

            # Применяем преобразования для улучшения читаемости имени
            # Пример: удаление ненужных символов, замена пробелов на нижнее подчеркивание и приведение к нижнему регистру
            new_base_name = re.sub(r'[^\w\s]', '', base_name)  # Убираем все спецсимволы
            new_base_name = new_base_name.replace(" ", "_")  # Заменяем пробелы на подчеркивания
            new_base_name = new_base_name.lower()  # Приводим к нижнему регистру

            # Добавление метки времени (например, если у вас есть несколько версий одной презентации)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            new_name = f"{new_base_name}_{timestamp}.pdf"

            # Новый путь для файла
            new_path = os.path.join(directory, new_name)

            # Переименование файла
            os.rename(old_path, new_path)
            print(f"Файл '{filename}' переименован в '{new_name}'")

# Пример использования
rename_presentation_files(raw_path)


Файл '2426HG2RXRZNRHX3YHORDF5C57PPAAHE (1).pdf' переименован в '2426hg2rxrznrhx3yhordf5c57ppaahe_1_20241213_153920.pdf'
Файл '2NYYOQN3UKYWMKXDAMTWP4ZNMGBBTLCA.pdf' переименован в '2nyyoqn3ukywmkxdamtwp4znmgbbtlca_20241213_153920.pdf'
Файл 'business-policy.pdf' переименован в 'businesspolicy_20241213_153920.pdf'
Файл '2IOECB7XKI56SDSRDXHXTAUDGCMYTVI7.pdf' переименован в '2ioecb7xki56sdsrdxhxtaudgcmytvi7_20241213_153920.pdf'
Файл '2L5JFJNVCBULD4V5UFNMPUKWM3BYSSYC (1).pdf' переименован в '2l5jfjnvcbuld4v5ufnmpukwm3byssyc_1_20241213_153920.pdf'
Файл '2C45NWO7FMTDRTBBR6FA442OZKSCLGZZ.pdf' переименован в '2c45nwo7fmtdrtbbr6fa442ozksclgzz_20241213_153920.pdf'
Файл '2426HG2RXRZNRHX3YHORDF5C57PPAAHE (1) (1).pdf' переименован в '2426hg2rxrznrhx3yhordf5c57ppaahe_1_1_20241213_153920.pdf'
Файл '25OOF5WZSJ4M6JG3BPYDO56GYBR65WET.pdf' переименован в '25oof5wzsj4m6jg3bpydo56gybr65wet_20241213_153920.pdf'
Файл '27CRLC3DDVOHBJ27UZSHYXRO3OB4BHIY.pdf' переименован в '27crlc3ddvohbj27uzshyxro3ob4bhiy_202412

In [ ]:
!pip install PyPDF2 pdfplumber pillow pandas


## Анализ содержимого документов

Разметка данным по


*   Шрифт
*   Количество картинок
*   Количество таблиц
*   Дизайн



In [ ]:
import fitz  # PyMuPDF
import pdfplumber
import os
import pandas as pd


def clean_table(table):
    # Очищаем таблицу от строк, которые состоят только из пустых значений
    cleaned_table = []
    for row in table:
        # Убираем пустые значения в строках
        cleaned_row = [value for value in row if value not in (None, '')]
        # Добавляем строку в таблицу, если она не пустая
        if cleaned_row:
            cleaned_table.append(cleaned_row)
    return cleaned_table

def extract_pdf_data(pdf_path):
    # Открытие PDF с помощью PyMuPDF для извлечения шрифтов и изображений
    doc = fitz.open(pdf_path)
    total_images = 0
    fonts = set()

    # Перебор всех страниц в документе для анализа шрифтов и изображений
    for page_num in range(doc.page_count):
        page = doc.load_page(page_num)

        # Подсчёт изображений на странице
        images = page.get_images(full=True)
        total_images += len(images)

        # Анализ шрифтов
        for text_instance in page.get_text("dict")["blocks"]:
            if text_instance['type'] == 0:  # Это текстовый блок
                for line in text_instance["lines"]:
                    for span in line["spans"]:
                        fonts.add(span['font'])

    # Используем pdfplumber для поиска таблиц
    total_tables = 0
    with pdfplumber.open(pdf_path) as pdf:
      for page in pdf.pages:
          # Извлечение таблиц на странице
          tables = page.extract_tables()
          if tables:
              print(tables)
              for table in tables:
                  if table:
                      cleaned_table = clean_table(table)
                      if cleaned_table:
                          total_tables += 1

    # Возвращаем собранную информацию
    data = {
        "font": ', '.join(fonts),
        "images_count": total_images,
        "tables_count": total_tables,
        "Design": "Не определён",  # Можно добавить более сложный анализ дизайна
    }
    return data

def process_pdf_files(directory_path):
    # Список для хранения данных
    all_data = []

    # Получаем все PDF файлы из папки
    for filename in os.listdir(directory_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(directory_path, filename)
            print(f"Обработка: {filename}")
            data = extract_pdf_data(pdf_path)
            # Добавляем имя файла в результат
            data["PDF файл"] = filename
            all_data.append(data)

    # Создание DataFrame
    df = pd.DataFrame(all_data)
    return df

# Пример использования
directory_path = raw_path  # Папка с вашими PDF файлами
df = process_pdf_files(directory_path)

# Вывод таблицы
print(df)

# Сохранение таблицы в CSV
df.to_csv("pdf_analysis_output_combined.csv", index=False)


Обработка: 27crlc3ddvohbj27uzshyxro3ob4bhiy_20241213_124515.pdf
Обработка: 2l5jfjnvcbuld4v5ufnmpukwm3byssyc_1_20241213_124515.pdf
[[['National TB Program\nObjectives', 'National TB Program\nIndicators'], ['•\nIncrease % of TB pts who\ncomplete TB treatment within 12\nmos.\n•\nIncrease % of TB pts with initial\npositive cultures who also are\ntested and receive drug\nsusceptibility results\n•\nDecrease the TB case rate in\nreporting areas\n•\nAt least 90% of core RVCT data\nare complete.', '•\nCompletion of therapy\nrates\n•\nPercent of cultured\nconfirmed drug cases with\nsusceptibility results\n•\nTB case rate\n•\nCompleteness of RVCT\nreporting']]]
Очищенная таблица: [['National TB Program\nObjectives', 'National TB Program\nIndicators'], ['•\nIncrease % of TB pts who\ncomplete TB treatment within 12\nmos.\n•\nIncrease % of TB pts with initial\npositive cultures who also are\ntested and receive drug\nsusceptibility results\n•\nDecrease the TB case rate in\nreporting areas\n•\nAt leas

In [ ]:
!pip install pdf2image openai


In [ ]:
!apt-get install -y poppler-utils


## Использование LLM c vision - gpt-4o-mini для разметки



*   Стиля текста (научный, популярный и т.д.)
*   Стиля презентации (формальная, монохромная и т.д.)

Проверка работы API






In [ ]:
from pdf2image import convert_from_path
import base64
import openai

# Настройка OpenAI API
openai.api_key = "xxxxx"

# Кодирование в BASE64
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Функция для конвертации PDF в изображения
def pdf_to_images(pdf_path):
    images = convert_from_path(pdf_path)
    return images

# Преобразование PDF в изображения
pdf_path = "/content/raw_data/2426hg2rxrznrhx3yhordf5c57ppaahe_1_1_20241213_124515.pdf"
images = pdf_to_images(pdf_path)

# Сохраняем изображения в виде файлов
image_paths = []
for i, image in enumerate(images):
    image_path = f"page_{i+1}.png"
    image.save(image_path, 'PNG')
    image_paths.append(image_path)

base64_image = encode_image(image_path)


In [ ]:
client = OpenAI(
    api_key="sk-or-vv-0dd695d9a77bd9c228d2b949cc6a53e17b48bc2f3696674f21f4eaaffb3adbf7", # ваш ключ в VseGPT после регистрации
    base_url="https://api.vsegpt.ru/v1",
)

# Path to your image
image_path = "path_to_your_image.jpg"

# Getting the base64 string

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "What is in this image?",
        },
        {
          "type": "image_url",
          "image_url": {
            "url":  f"data:image/jpeg;base64,{base64_image}"
          },
        },
      ],
    }
  ],
)

print(response.choices[0])


Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The image presents a diagram titled "New Jersey History of Oxygenated Fuel." It illustrates different percentages related to the use of oxygenated fuel in New Jersey over various time periods, focusing on regulations and mandates. Key points include:\n\n- **New Jersey CO SIP Rule:** Shows a range of 1.7% to 2.7% for some periods.\n- **USEPA Rule or Non-mandated Usage:** Indicates a range of 0.8% to 1.0% for some fuel providers in the 1980s, and mentions a percentage of 2.0% for RFG I in 1995.\n- **Comparative Differences:** Highlights a difference of 0.7% in certain usages.\n\nThe timeline spans from the 1980s to 1995, marking changes in regulations over the years.', refusal='', role='assistant', audio=None, function_call=None, tool_calls=None))


In [ ]:
%pip install python-dotenv

## Скрипт разметки данных

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Настройка OpenAI API
openai.api_key = os.getenv('OPENAI_KEY')

# Функция для кодирования изображения в base64
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Функция для конвертации PDF в изображения
def pdf_to_images(pdf_path):
    images = convert_from_path(pdf_path, fmt='jpeg')  # Сохраняем изображения как JPEG
    return images

# Функция для обработки PDF файлов в директории
def process_pdfs_in_directory(raw_path):
    image_paths = []
    # Получаем все файлы в директории
    for filename in os.listdir(raw_path):
        if filename.endswith(".pdf"):
            pdf_path = os.path.join(raw_path, filename)
            images = pdf_to_images(pdf_path)

            # Сохраняем каждое изображение и получаем пути
            for i, image in enumerate(images):
                image_path = f"{pdf_path}_page_{i+1}.jpg"
                image.save(image_path, 'JPEG')  # Сохраняем в JPEG
                image_paths.append(image_path)
    return image_paths

# Директория с PDF файлами
raw_path = "/content/raw_data"

# Обработка всех PDF файлов в директории
image_paths = process_pdfs_in_directory(raw_path)

# Отправка изображений в OpenAI
for image_path in image_paths:
    base64_image = encode_image(image_path)

    # Запрос к OpenAI для анализа изображения
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": "What is the style of this presentation (formal, scientific, etc.) and its design style in words?"},
                    {
                        "type": "image_url",
                        "image_url": f"data:image/jpeg;base64,{base64_image}"
                    },
                ]
            }
        ]
    )

    print(f"Response for {image_path}: {response.choices[0].message['content']}")


BadRequestError: Error code: 400 - {'error': {'message': 'Potentially out of budget: 237285->700, expected price 4.8017, but you have only 1.107680 on account. Please, add some money to balance to proceed: https://vsegpt.ru/User/Money', 'code': 400}}